# Fase 02 — prepareEventsDS (notebook)

Este notebook genera el **dataset de eventos** a partir del dataset limpio
producido en la Fase 01 (`01_explore`), siguiendo el mismo patrón de variantes
y trazabilidad que en la Fase 01.

**Entradas principales:**
- Dataset explorado de la Fase 01, asociado a una variante padre.
- `Tu` (paso temporal característico) obtenido de la metadata de la Fase 01
  o, en su defecto, del bloque `global.Tu` en `params.yaml`.
- Parámetros de Fase 02 (por variante):
    - `band_thresholds_pct` — lista de porcentajes de corte sobre el rango.
    - `event_strategy` ∈ {`levels`, `transitions`, `both`}.
    - `nan_handling` ∈ {`keep`, `drop`} (por ahora).

**Salidas principales:**
- Catálogo de eventos: `outputs/metadata/02_event_dict.json` (nombre → ID).
- Estadísticos min/max por medida: `outputs/metadata/02_minmax_stats.json`.
- Dataset de eventos:
    - `data/02-interim/02_events_dataset.parquet`
    - `outputs/interim/02_events_dataset.parquet`
- Figuras en `outputs/figures/`:
    - `02_event_counts.png`
    - `02_event_counts_by_measure.png`
    - `02_event_interarrival_<id>.png`
- Informe HTML: `outputs/reports/02_events_summary.html`.

⚠️ **No crea variantes nuevas**.  
Las variantes se crean SIEMPRE desde la línea de comandos con:

```bash
make variant2 VARIANT=v011 RAW=/ruta/dataset.csv     CLEANING_STRATEGY=basic     NAN_VALUES='[-999999,-1]'     ERROR_VALUES='{"hum":[0,999],"temp":[-50]}'
```

Y el notebook se ejecuta para una variante con:

```bash
make nb2-run VARIANT=v011
```


In [1]:
# =====================================================================
# 1. IMPORTS COMUNES
# =====================================================================
import os
import sys
import json
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timezone
import seaborn as sns
import shutil

# Estilo gráfico común
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

print("✔ Imports y estilo cargados")

✔ Imports y estilo cargados


In [2]:
# =====================================================================
# 2. CONTEXTO DE EJECUCIÓN (bootstrap + run_context)
# =====================================================================
import sys
from pathlib import Path

# --- BOOTSTRAP MÍNIMO (antes de cualquier import mlops4ofp) ---
execution_dir = Path.cwd().resolve()

current = execution_dir
for _ in range(10):
    if (current / "mlops4ofp").exists():
        project_root = current
        break
    current = current.parent
else:
    raise RuntimeError("❌ No se pudo localizar project_root")

sys.path.insert(0, str(project_root))

print(f"📁 Project root añadido a PYTHONPATH: {project_root}")

# --- AHORA SÍ: imports normales ---
from mlops4ofp.tools.run_context import (
    detect_execution_dir,
    detect_project_root,
    assemble_run_context,
)
import yaml

PHASE = "02_prepareeventsds"
phase = PHASE

# -------------------------------------------------------------------
# Selección de variante
# -------------------------------------------------------------------
ACTIVE_VARIANT = os.environ.get("ACTIVE_VARIANT")

# Caso 1: ejecución vía Makefile (preferido)
if ACTIVE_VARIANT:
    print(f"[INFO] Variante activa (desde entorno): {ACTIVE_VARIANT}")

# Caso 2: notebook abierto manualmente
else:
    print("⚠️ ACTIVE_VARIANT no definido — seleccionando variante automáticamente…")

    variants_file = project_root / "executions" / PHASE / "variants.yaml"

    if not variants_file.exists():
        raise RuntimeError(
            "❌ No existen variantes todavía.\n"
            "Crea una con:\n"
            "    make variant1 VARIANT=v001 RAW=tu_dataset.csv"
        )

    with open(variants_file, "r", encoding="utf-8") as f:
        reg = yaml.safe_load(f) or {}

    all_variants = sorted(reg.get("variants", {}).keys())
    if not all_variants:
        raise RuntimeError("❌ No hay variantes definidas en variants.yaml")

    ACTIVE_VARIANT = all_variants[-1]
    print(f"[INFO] Usando variante más reciente: {ACTIVE_VARIANT}")

# -------------------------------------------------------------------
# Construcción del contexto final
# -------------------------------------------------------------------
from mlops4ofp.tools.params_manager import ParamsManager

pm = ParamsManager(PHASE, project_root)

print("PHASE_DIR según ParamsManager:", pm.phase_dir)
print("¿Existe PHASE_DIR?", pm.phase_dir.exists())
print("¿Existe variante?", (pm.phase_dir / ACTIVE_VARIANT).exists())

pm.set_current(ACTIVE_VARIANT)

variant_root = pm.current_variant_dir()

ctx = assemble_run_context(
    execution_dir=detect_execution_dir(),
    project_root=project_root,
    phase=PHASE,
    variant=ACTIVE_VARIANT,
    variant_root=variant_root,
)

print("✔ Contexto de ejecución construido")
print(f"   Fase: {ctx['phase']}")
print(f"   Variante: {ctx['variant']}")
print(f"   Carpeta variante: {ctx['variant_root']}")


📁 Project root añadido a PYTHONPATH: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp
[INFO] Variante activa (desde entorno): v011
PHASE_DIR según ParamsManager: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds
¿Existe PHASE_DIR? True
¿Existe variante? True
✔ Contexto de ejecución construido
   Fase: 02_prepareeventsds
   Variante: v011
   Carpeta variante: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011


In [3]:
# =====================================================================
# 4. RUTAS DE DATOS Y SALIDAS (VARIANTE)
#    Todos los ficheros llevan como prefijo el nombre de la fase
# =====================================================================
from mlops4ofp.tools.run_context import build_phase_outputs
VARIANT_DIR = ctx["variant_root"]
PHASE_PREFIX = ctx["phase"]

# Datos de entrada (raw es compartido por todo el proyecto)
RAW_DIR = ctx["project_root"] / "data" / "01-raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUTS = build_phase_outputs(
    variant_root=VARIANT_DIR,
    phase=ctx["phase"],
)

# Figuras (única subcarpeta permitida dentro de la variante)
FIGURES_DIR = ctx["figures_dir"]

print("✔ Rutas de salida preparadas (scoped a la variante)")
OUTPUTS, FIGURES_DIR

✔ Rutas de salida preparadas (scoped a la variante)


({'dataset': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_dataset.parquet'),
  'report': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_report.html'),
  'params': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_params.json'),
  'metadata': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_metadata.json')},
 PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/figures'))

In [4]:
# =====================================================================
# 3. Leer parámetros de la variante activa (Fase 02)
# =====================================================================
from mlops4ofp.tools.params_manager import validate_params

params_path = VARIANT_DIR / "params.yaml"
if not params_path.exists():
    raise FileNotFoundError(
        f"No existe el fichero de parámetros de la variante: {params_path}\n"
        f"Primero crea la variante con:\n"
        f"    make variant2 VARIANT={ACTIVE_VARIANT} ..."
    )

# Leer parámetros + validar contra traceability_schema.yaml
with open(params_path, "r", encoding="utf-8") as f:
    params_f02 = yaml.safe_load(f) or {}

validate_params(PHASE, params_f02, ctx["project_root"])
print("✔ Parámetros de Fase 02 validados correctamente según schema.")

# ---------------------------------------------------------------------
# Parámetros propios de Fase 02
# ---------------------------------------------------------------------
band_thresholds_pct = params_f02.get("band_thresholds_pct", [40, 60, 90])
event_strategy = params_f02.get("event_strategy", "both")
nan_handling = params_f02.get("nan_handling", "keep")

parent_phase = params_f02.get("parent_phase", "01_explore")
parent_variant = params_f02.get("parent_variant")
variant_id = params_f02.get("variant_id", ACTIVE_VARIANT)

# Validaciones mínimas explícitas (fase-específicas)
ALLOWED_STRATEGIES = {"transitions", "levels", "both"}
if event_strategy not in ALLOWED_STRATEGIES:
    raise ValueError(
        f"event_strategy='{event_strategy}' no es válida.\n"
        f"Opciones permitidas: {ALLOWED_STRATEGIES}"
    )

ALLOWED_NAN_HANDLING = {"keep", "drop"}
if nan_handling not in ALLOWED_NAN_HANDLING:
    raise ValueError(
        f"nan_handling='{nan_handling}' no es válida.\n"
        f"Opciones permitidas: {ALLOWED_NAN_HANDLING}"
    )

# Log compacto
print("────────── VARIANT PARAMETERS (F02) ──────────")
print(f"variant_id           = {variant_id}")
print(f"parent_phase         = {parent_phase}")
print(f"parent_variant       = {parent_variant}")
print(f"band_thresholds_pct  = {band_thresholds_pct}")
print(f"event_strategy       = {event_strategy}")
print(f"nan_handling         = {nan_handling}")
print("──────────────────────────────────────────────")

# =====================================================================
# 4. Cargar metadata de la Fase 01 (padre) para herencia de Tu
# =====================================================================
parent_metadata_path = (
    ctx["project_root"]
    / "executions"
    / parent_phase
    / parent_variant
    / f"{parent_phase}_metadata.json"
)

if not parent_metadata_path.exists():
    raise FileNotFoundError(
        f"No se encontró metadata de la fase padre:\n{parent_metadata_path}"
    )

with open(parent_metadata_path, "r", encoding="utf-8") as f:
    parent_params_f01 = json.load(f)

# =====================================================================
# 5. Resolver Tu (prioridad: F02 → F01)
# =====================================================================
def resolve_Tu(params_f02, parent_params_f01):
    """
    Selección de Tu:
      1. Explícito en la variante de Fase 02
      2. Heredado de metadata de Fase 01
      3. Error si no existe
    """
    if params_f02.get("Tu") is not None:
        Tu = float(params_f02["Tu"])
        print(f"✔ Tu tomado de variante Fase 02: {Tu}")
        return Tu

    Tu_f01 = parent_params_f01.get("Tu")
    if Tu_f01 is not None:
        Tu = float(Tu_f01)
        print(f"✔ Tu heredado de metadata Fase 01: {Tu}")
        return Tu

    raise RuntimeError("❌ No se pudo determinar Tu (ni en Fase 02 ni en Fase 01).")


Tu = resolve_Tu(params_f02, parent_params_f01)
print(f"👉 Tu final = {Tu} segundos")



✔ Parámetros de Fase 02 validados correctamente según schema.
────────── VARIANT PARAMETERS (F02) ──────────
variant_id           = v011
parent_phase         = 01_explore
parent_variant       = v001
band_thresholds_pct  = [5, 15, 25, 40, 60, 75, 85, 95]
event_strategy       = transitions
nan_handling         = keep
──────────────────────────────────────────────
✔ Tu heredado de metadata Fase 01: 10.0
👉 Tu final = 10.0 segundos


In [5]:
# =====================================================================
# 6. Cargar dataset de entrada (resultado de Fase 01)
# =====================================================================

parent_dataset_path = (
    ctx["project_root"]
    / "executions"
    / parent_phase
    / parent_variant
    / "01_explore_dataset.parquet"
)

if not parent_dataset_path.exists():
    raise FileNotFoundError(
        f"No se encontró el dataset explorado de la fase padre:\n{parent_dataset_path}"
    )

print(f"✔ Dataset padre encontrado:\n{parent_dataset_path}")

df_explored = pd.read_parquet(parent_dataset_path)

print("Shape dataset explorado:", df_explored.shape)
display(df_explored.head())


✔ Dataset padre encontrado:
/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/01_explore/v001/01_explore_dataset.parquet


Shape dataset explorado: (3887242, 18)


,segs,Battery_Active_Power,Battery_Active_Power_Set_Response,PVPCS_Active_Power,GE_Body_Active_Power,GE_Active_Power,GE_Body_Active_Power_Set_Response,FC_Active_Power_FC_END_Set,FC_Active_Power,FC_Active_Power_FC_end_Set_Response,Island_mode_MCCB_Active_Power,MG-LV-MSB_AC_Voltage,Receiving_Point_AC_Voltage,Island_mode_MCCB_AC_Voltage,Island_mode_MCCB_Frequency,MG-LV-MSB_Frequency,Inlet_Temperature_of_Chilled_Water,Outlet_Temperature
0,1651363201,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000
1,1651363211,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000
2,1651363221,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-13.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.200001
3,1651363231,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-13.0,482.0,483.0,482.0,59.950001,59.950001,22.1,22.200001
4,1651363241,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000


In [6]:
# =====================================================================
# 7. Normalización temporal y columnas base
# =====================================================================

# --- 7.1 Comprobación de columna temporal ---
if "segs" not in df_explored.columns:
    raise RuntimeError(
        "❌ El dataset explorado no contiene la columna temporal obligatoria 'segs'."
    )

# Asegurar orden temporal
df_explored = df_explored.sort_values("segs").reset_index(drop=True)

# --- 7.2 Eliminar Timestamp si existe (ya no es necesario en F02) ---
if "Timestamp" in df_explored.columns:
    df_explored = df_explored.drop(columns=["Timestamp"])
    print("✔ Columna 'Timestamp' eliminada")

# --- 7.3 Identificación de columnas de medida ---
measurement_cols = [
    c for c in df_explored.columns
    if c != "segs"
]

if not measurement_cols:
    raise RuntimeError("❌ No se han detectado columnas de medida.")

print(f"✔ Columnas de medida detectadas ({len(measurement_cols)}):")
print(measurement_cols)

# Vista rápida
display(df_explored.head())


✔ Columnas de medida detectadas (17):
['Battery_Active_Power', 'Battery_Active_Power_Set_Response', 'PVPCS_Active_Power', 'GE_Body_Active_Power', 'GE_Active_Power', 'GE_Body_Active_Power_Set_Response', 'FC_Active_Power_FC_END_Set', 'FC_Active_Power', 'FC_Active_Power_FC_end_Set_Response', 'Island_mode_MCCB_Active_Power', 'MG-LV-MSB_AC_Voltage', 'Receiving_Point_AC_Voltage', 'Island_mode_MCCB_AC_Voltage', 'Island_mode_MCCB_Frequency', 'MG-LV-MSB_Frequency', 'Inlet_Temperature_of_Chilled_Water', 'Outlet_Temperature']


,segs,Battery_Active_Power,Battery_Active_Power_Set_Response,PVPCS_Active_Power,GE_Body_Active_Power,GE_Active_Power,GE_Body_Active_Power_Set_Response,FC_Active_Power_FC_END_Set,FC_Active_Power,FC_Active_Power_FC_end_Set_Response,Island_mode_MCCB_Active_Power,MG-LV-MSB_AC_Voltage,Receiving_Point_AC_Voltage,Island_mode_MCCB_AC_Voltage,Island_mode_MCCB_Frequency,MG-LV-MSB_Frequency,Inlet_Temperature_of_Chilled_Water,Outlet_Temperature
0,1651363201,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000
1,1651363211,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000
2,1651363221,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-13.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.200001
3,1651363231,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-13.0,482.0,483.0,482.0,59.950001,59.950001,22.1,22.200001
4,1651363241,0.0,0.0,0.0,0.0,-1.0,180.0,40.0,38.0,40.0,-14.0,482.0,483.0,482.0,59.959999,59.959999,22.1,22.100000


In [7]:
# ============================================================
# FUNCIONES AUXILIARES (Fase 02 – optimizadas y rápidas)
# ============================================================

import numpy as np
import pandas as pd
from typing import Dict, List
from pathlib import Path


# ------------------------------------------------------------
# Bandas: cortes y etiquetas (vectorizado)
# ------------------------------------------------------------

def compute_cuts_and_labels(
    minmax_stats: Dict[str, Dict[str, float]],
    pct_thresholds: List[float],
) -> Dict[str, Dict[str, np.ndarray]]:
    """
    Devuelve para cada variable:
      {
        col: {
          "cuts": np.ndarray,
          "labels": np.ndarray(dtype=object)
        }
      }
    """
    pct_list = np.array([0.0] + pct_thresholds + [100.0])
    out = {}

    for col, mm in minmax_stats.items():
        mn = mm["min"]
        mx = mm["max"]
        r = mx - mn

        if r == 0.0:
            cuts = np.array([mn, mx])
            labels = np.array(["0_100"], dtype=object)
        else:
            cuts = mn + (pct_list / 100.0) * r
            labels = np.array(
                [f"{int(pct_list[i])}_{int(pct_list[i+1])}"
                 for i in range(len(pct_list) - 1)],
                dtype=object
            )

        out[col] = {"cuts": cuts, "labels": labels}

    return out


# ------------------------------------------------------------
# Asignación vectorizada a bandas (muy rápida)
# ------------------------------------------------------------

def assign_bands_to_column(values: np.ndarray, cuts: np.ndarray, labels: np.ndarray):
    """
    Para un vector de valores numéricos, devuelve:
        kind  : array {"none","nan","band"}
        label : array con "xx_yy" o None
    """
    is_nan = np.isnan(values)
    kind = np.where(is_nan, "NaN", "band")

    idx = np.searchsorted(cuts, values, side="right") - 1
    idx = np.clip(idx, 0, len(labels) - 1)

    assigned_labels = labels[idx].copy()
    assigned_labels[is_nan] = None

    out_of_range = (values < cuts[0]) | (values > cuts[-1])
    assigned_labels[out_of_range] = None
    kind[out_of_range] = "none"

    return kind, assigned_labels


# ------------------------------------------------------------
# Catálogo de eventos (IDs compactos)
# ------------------------------------------------------------

def build_event_catalog(bands, event_strategy, nan_handling):
    """
    Devuelve:
      { event_name → event_id }
    """
    event_to_id = {}
    next_id = 1

    strat = event_strategy.lower()
    nan_keep = (nan_handling.lower() == "keep")

    for col, info in bands.items():
        labels = info["labels"]

        # Eventos de transición (solo band → band)
        if strat in ("transitions", "both"):
            for a in labels:
                for b in labels:
                    if a != b:
                        event_to_id[f"{col}_{a}-to-{b}"] = next_id
                        next_id += 1

        # Eventos de nivel (band)
        if strat in ("levels", "both"):
            for a in labels:
                event_to_id[f"{col}_{a}"] = next_id
                next_id += 1

        # Evento de nivel NaN (independiente de la estrategia)
        if nan_keep:
            event_to_id[f"{col}_NaN_NaN"] = next_id
            next_id += 1

    return event_to_id


# ------------------------------------------------------------
# Generación rápida de eventos (núcleo F02)
# ------------------------------------------------------------

def fast_generate_events(
    df,
    measure_cols,
    bands,
    event_to_id,
    event_strategy,
    nan_handling,
    Tu,
):
    """
    Generación eficiente del dataset final de eventos.
    """
    N = len(df)
    segs = df["segs"].values.astype(np.int64)

    is_consecutive = np.zeros(N, dtype=bool)
    is_consecutive[1:] = (np.diff(segs) == Tu)

    strat = event_strategy.lower()
    nan_keep = (nan_handling.lower() == "keep")

    events_column = [[] for _ in range(N)]

    prev_kind = {col: None for col in measure_cols}
    prev_label = {col: None for col in measure_cols}

    col_kind = {}
    col_label = {}

    for col in measure_cols:
        vals = df[col].values
        info = bands[col]
        k_arr, lbl_arr = assign_bands_to_column(vals, info["cuts"], info["labels"])
        k_arr = np.where(k_arr == "NaN", "NaN", k_arr)
        col_kind[col] = k_arr
        col_label[col] = lbl_arr

    for i in range(N):
        row_events = []

        for col in measure_cols:
            curr_k = col_kind[col][i]
            curr_lbl = col_label[col][i]

            # Transiciones
            if i > 0 and is_consecutive[i] and strat in ("transitions", "both"):
                pk = prev_kind[col]
                pl = prev_label[col]

                if pk == "band" and curr_k == "band" and pl != curr_lbl:
                    ev = event_to_id.get(f"{col}_{pl}-to-{curr_lbl}")
                    if ev:
                        row_events.append(ev)

            # NIVELES
            if curr_k == "band" and strat in ("levels", "both"):
                ev = event_to_id.get(f"{col}_{curr_lbl}")
                if ev:
                    row_events.append(ev)

            elif curr_k == "NaN" and nan_keep:
                ev = event_to_id.get(f"{col}_NaN_NaN")
                if ev:
                    row_events.append(ev)

            prev_kind[col] = curr_k
            prev_label[col] = curr_lbl

        events_column[i] = row_events

    return pd.DataFrame({
        "segs": segs,
        "events": events_column,
    })


# ------------------------------------------------------------
# Min / Max por medida (Fase 02)
# ------------------------------------------------------------

def compute_minmax(df: pd.DataFrame, measure_cols: List[str]) -> Dict[str, Dict[str, float]]:
    return {
        col: {
            "min": float(df[col].min()),
            "max": float(df[col].max()),
        }
        for col in measure_cols
    }


minmax_stats = compute_minmax(df_explored, measurement_cols)

print("[prepareeventsds] min/max por medida calculado.")
for c, mm in list(minmax_stats.items())[:5]:
    print(f"  {c}: min={mm['min']:.3f}, max={mm['max']:.3f}")




[prepareeventsds] min/max por medida calculado.
  Battery_Active_Power: min=-216.800, max=119.900
  Battery_Active_Power_Set_Response: min=-90.000, max=100.000
  PVPCS_Active_Power: min=-3.000, max=51.000
  GE_Body_Active_Power: min=-10.000, max=293.000
  GE_Active_Power: min=-601.000, max=400.000


In [8]:
# =====================================================================
# 8. Pipeline de generación de eventos (Fase 02)
# =====================================================================

# --- 8.1 Construcción de bandas ---
bands = compute_cuts_and_labels(
    minmax_stats=minmax_stats,
    pct_thresholds=band_thresholds_pct,
)

print(f"✔ Bandas construidas para {len(bands)} medidas")

# Ejemplo de bandas (primeras 2 columnas)
for c in list(bands.keys())[:2]:
    print(f"{c}:")
    print("  cuts  :", bands[c]["cuts"])
    print("  labels:", bands[c]["labels"])


# --- 8.2 Catálogo de eventos ---
event_to_id = build_event_catalog(
    bands=bands,
    event_strategy=event_strategy,
    nan_handling=nan_handling,
)

print(f"✔ Catálogo de eventos generado: {len(event_to_id)} eventos")


# --- 8.3 Generación del dataset de eventos ---
df_events = fast_generate_events(
    df=df_explored,
    measure_cols=measurement_cols,
    bands=bands,
    event_to_id=event_to_id,
    event_strategy=event_strategy,
    nan_handling=nan_handling,
    Tu=Tu,
)

# =====================================================================
# VERIFICACIÓN EVENTOS NaN (F02)
# =====================================================================

nan_event_ids = {
    eid for name, eid in event_to_id.items()
    if name.endswith("_NaN_NaN")
}

num_rows_with_nan_event = sum(
    any(e in nan_event_ids for e in evs)
    for evs in df_events["events"]
)

print("Filas con eventos NaN (nivel):", num_rows_with_nan_event)
print("IDs de eventos NaN:", nan_event_ids)



print("✔ Dataset de eventos generado")
print("Shape:", df_events.shape)

display(df_events.head())


✔ Bandas construidas para 17 medidas
Battery_Active_Power:
  cuts  : [-216.800003   -199.96500275 -166.29500225 -132.62500175  -82.120001
  -14.78         35.72500075   69.39500125  103.06500175  119.900002  ]
  labels: ['0_5' '5_15' '15_25' '25_40' '40_60' '60_75' '75_85' '85_95' '95_100']
Battery_Active_Power_Set_Response:
  cuts  : [-90.  -80.5 -61.5 -42.5 -14.   24.   52.5  71.5  90.5 100. ]
  labels: ['0_5' '5_15' '15_25' '25_40' '40_60' '60_75' '75_85' '85_95' '95_100']
✔ Catálogo de eventos generado: 1241 eventos


Filas con eventos NaN (nivel): 2571
IDs de eventos NaN: {803, 292, 1095, 584, 73, 876, 365, 1168, 657, 146, 949, 438, 1241, 730, 219, 1022, 511}
✔ Dataset de eventos generado
Shape: (3887242, 2)


,segs,events
0,1651363201,[]
1,1651363211,[]
2,1651363221,[]
3,1651363231,[]
4,1651363241,[]


In [9]:
# =====================================================================
# 9. Persistencia de resultados, metadata y trazabilidad (Fase 02)
# =====================================================================

import json
from mlops4ofp.tools.artifacts import (
    get_git_hash,
    save_numeric_dataset,
    save_params_and_metadata,
)

# ---------------------------------------------------------------------
# 9.1 Rutas de salida (autocontenidas por fase / variante)
# ---------------------------------------------------------------------
PHASE_DIR = ctx["variant_root"]

# ---------------------------------------------------------------------
# 9.2 Guardar dataset de eventos
# ---------------------------------------------------------------------
events_dataset_path = PHASE_DIR / "02_prepareeventsds_dataset.parquet"


# DIAGNÓSTICO ROBUSTO DE EVENTOS NaN EN F02
def has_nan_event(evs):
    if evs is None:
        return False

    # Evento único como string
    if isinstance(evs, str):
        return "nan" in evs.lower()

    # Lista / iterable de eventos
    if isinstance(evs, (list, tuple, set)):
        for e in evs:
            if isinstance(e, str) and "nan" in e.lower():
                return True
        return False

    # Cualquier otro tipo (int, float, etc.)
    return False

print("Tipos distintos en df['events']:",
      {type(evs).__name__ for evs in df_events["events"]})

print("Timestamps con eventos NaN:",
      sum(has_nan_event(evs) for evs in df_events["events"]))

# Dataset de eventos: columna temporal + lista de eventos
df_events.to_parquet(events_dataset_path, index=False)

print(f"[OK] Dataset de eventos guardado en: {events_dataset_path}")

# ---------------------------------------------------------------------
# 9.3 Guardar artefactos auxiliares (bandas y catálogo)
# ---------------------------------------------------------------------
bands_path = PHASE_DIR / "02_prepareeventsds_bands.json"
event_catalog_path = PHASE_DIR / "02_prepareeventsds_event_catalog.json"

with open(bands_path, "w", encoding="utf-8") as f:
    json.dump(
        {
            col: {
                "cuts": bands[col]["cuts"].tolist(),
                "labels": bands[col]["labels"].tolist(),
            }
            for col in bands
        },
        f,
        indent=2,
    )

with open(event_catalog_path, "w", encoding="utf-8") as f:
    json.dump(event_to_id, f, indent=2)

print("[OK] Artefactos guardados:")
print(f"  - {bands_path}")
print(f"  - {event_catalog_path}")

# ---------------------------------------------------------------------
# 9.4 Parámetros generados (para trazabilidad)
# ---------------------------------------------------------------------
gen_params = {
    "Tu": float(Tu),
    "band_thresholds_pct": band_thresholds_pct,
    "event_strategy": event_strategy,
    "nan_handling": nan_handling,
    "n_rows_input": int(len(df_explored)),
    "n_rows_events": int(len(df_events)),
    "n_events_catalog": int(len(event_to_id)),
    "n_measures": int(len(measurement_cols)),
}

# ---------------------------------------------------------------------
# 9.5 Metadata Fase 02 (extendida)
# ---------------------------------------------------------------------
metadata_extra = {
    "dataset_events": str(events_dataset_path),
    "parent_phase": parent_phase,
    "parent_variant": parent_variant,
    "input_dataset": str(parent_dataset_path),
    "Tu": float(Tu),
    "band_thresholds_pct": band_thresholds_pct,
    "event_strategy": event_strategy,
    "nan_handling": nan_handling,
    "n_rows_input": int(len(df_explored)),
    "n_rows_events": int(len(df_events)),
    "n_events_catalog": int(len(event_to_id)),
    "n_measures": int(len(measurement_cols)),
}

# ---------------------------------------------------------------------
# 9.6 Guardar params + metadata + trazabilidad (como en F01)
# ---------------------------------------------------------------------
save_params_and_metadata(
    phase=PHASE,
    variant=ACTIVE_VARIANT,
    variant_root=ctx["variant_root"],
    raw_path=parent_dataset_path,   # dataset padre como "raw lógico"
    gen_params=gen_params,
    metadata_extra=metadata_extra,
    pm=pm,                          # ParamsManager (opcional)
    git_commit=get_git_hash(),
)

print("[OK] Metadata y trazabilidad Fase 02 registradas correctamente")



Tipos distintos en df['events']: {'list'}


Timestamps con eventos NaN: 0


[OK] Dataset de eventos guardado en: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_dataset.parquet
[OK] Artefactos guardados:
  - /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_bands.json
  - /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_event_catalog.json
[OK] Metadata y trazabilidad Fase 02 registradas correctamente


In [10]:
# =====================================================================
# 4. RUTAS DE DATOS Y SALIDAS (VARIANTE)
#    Todos los ficheros llevan como prefijo el nombre de la fase
# =====================================================================
from mlops4ofp.tools.run_context import build_phase_outputs
VARIANT_DIR = ctx["variant_root"]
PHASE_PREFIX = ctx["phase"]

# Datos de entrada (raw es compartido por todo el proyecto)
#RAW_DIR = ctx["project_root"] / "data" / "01-raw"
#RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUTS = build_phase_outputs(
    variant_root=VARIANT_DIR,
    phase=ctx["phase"],
)

# Figuras (única subcarpeta permitida dentro de la variante)
FIGURES_DIR = ctx["figures_dir"]

print("✔ Rutas de salida preparadas (scoped a la variante)")
OUTPUTS, FIGURES_DIR

✔ Rutas de salida preparadas (scoped a la variante)


({'dataset': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_dataset.parquet'),
  'report': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_report.html'),
  'params': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_params.json'),
  'metadata': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_metadata.json')},
 PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/figures'))

In [11]:
# ============================================================
# 10.x — Tablas, estadísticas y figuras (Fase 02)
# ============================================================

from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlops4ofp.tools.figures import save_figure

print("[prepareeventsds] Generando tablas, estadísticas y figuras...")

FIGURES_DIR = VARIANT_DIR / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

MAX_X_TU = 10
MAX_BINS = 10

# ------------------------------------------------------------
# Preparación común
# ------------------------------------------------------------

flat_events = [ev for evs in df_events["events"] for ev in evs]
counts = Counter(flat_events)
inv_event = {v: k for k, v in event_to_id.items()}

MEASURE_NAMES = sorted(measurement_cols, key=len, reverse=True)

def extract_measure(ev_name: str) -> str:
    for m in MEASURE_NAMES:
        if ev_name.startswith(m + "_") or ev_name == m:
            return m
    return ev_name.split("_")[0]

# ------------------------------------------------------------
# Tabla 1 — Recuento por tipo de evento
# ------------------------------------------------------------

df_event_counts = (
    pd.DataFrame([
        {"event_id": ev_id, "event_name": inv_event[ev_id], "count": cnt}
        for ev_id, cnt in counts.items()
    ])
    .sort_values("event_id")
    .reset_index(drop=True)
)

# ------------------------------------------------------------
# Tabla 2 — Recuento por medida
# ------------------------------------------------------------

measure_counts = {}
for ev_id, cnt in counts.items():
    m = extract_measure(inv_event[ev_id])
    measure_counts[m] = measure_counts.get(m, 0) + cnt

df_measure_counts = (
    pd.DataFrame(
        [{"measure": m, "count": c} for m, c in measure_counts.items()]
    )
    .sort_values("measure")
    .reset_index(drop=True)
)

# ------------------------------------------------------------
# Índice inverso: event_id → posiciones
# ------------------------------------------------------------

event_positions = {ev_id: [] for ev_id in event_to_id.values()}
for idx, evs in enumerate(df_events["events"].values):
    for ev in evs:
        event_positions[ev].append(idx)

for ev_id in event_positions:
    event_positions[ev_id] = np.array(event_positions[ev_id], dtype=np.int64)

# ------------------------------------------------------------
# Tabla 3 — Interarrival por evento (solo num_appearances > 0)
# ------------------------------------------------------------

interarrival_rows = []

for ev_id, positions in event_positions.items():
    if len(positions) == 0:
        continue

    epochs = df_events.iloc[positions]["segs"].values.astype(float)

    if len(epochs) < 2:
        interarrival_rows.append({
            "event_id": ev_id,
            "event_name": inv_event[ev_id],
            "num_appearances": len(epochs),
            "num_intervals": 0,
            "mean_interarrival": None,
            "std_interarrival": None,
            "min_interarrival": None,
            "max_interarrival": None,
        })
        continue

    deltas = np.diff(epochs)
    deltas = deltas[deltas >= 0]

    if len(deltas) == 0:
        interarrival_rows.append({
            "event_id": ev_id,
            "event_name": inv_event[ev_id],
            "num_appearances": len(epochs),
            "num_intervals": 0,
            "mean_interarrival": None,
            "std_interarrival": None,
            "min_interarrival": None,
            "max_interarrival": None,
        })
        continue

    interarrival_rows.append({
        "event_id": ev_id,
        "event_name": inv_event[ev_id],
        "num_appearances": len(epochs),
        "num_intervals": len(deltas),
        "mean_interarrival": float(np.mean(deltas)),
        "std_interarrival": float(np.std(deltas)),
        "min_interarrival": float(np.min(deltas)),
        "max_interarrival": float(np.max(deltas)),
    })

df_interarrival_stats = (
    pd.DataFrame(interarrival_rows)
    .sort_values("event_id")
    .reset_index(drop=True)
)

# ------------------------------------------------------------
# FIGURAS — Interarrival por EVENTO (en unidades Tu)
# ------------------------------------------------------------

interarrival_event_info = {}

for _, row in df_interarrival_stats.iterrows():
    ev_id = row["event_id"]
    ev_name = row["event_name"]
    positions = event_positions[ev_id]

    if len(positions) < 3:
        interarrival_event_info[ev_id] = {"has_fig": False, "reason": "<3 apariciones"}
        continue

    epochs = df_events.iloc[positions]["segs"].values.astype(float)
    deltas = np.diff(epochs)
    deltas = deltas[deltas >= 0]

    if len(deltas) < 2:
        interarrival_event_info[ev_id] = {"has_fig": False, "reason": "intervalos insuficientes"}
        continue

    deltas_TU = deltas / Tu
    clipped = np.clip(deltas_TU, 0, MAX_X_TU)

    fig_path = FIGURES_DIR / f"interarrival_event_{ev_id}.png"

    def _plot():
        bins = min(MAX_BINS, len(np.unique(clipped)))
        plt.hist(clipped, bins=bins, edgecolor="black")
        plt.xlabel("Tiempo entre llegadas (Tu)")
        plt.ylabel("count")
        plt.title(f"Interarrival — evento {ev_id}")
        plt.xlim(0, MAX_X_TU)
        if np.any(deltas_TU > MAX_X_TU):
            plt.annotate("→", xy=(MAX_X_TU, plt.ylim()[1]*0.9),
                         color="red", fontsize=20, ha="center")

    save_figure(fig_path, plot_fn=_plot, figsize=(10, 4))

    interarrival_event_info[ev_id] = {
        "has_fig": True,
        "file": fig_path.name,
        "num_app": int(len(epochs)),
    }

# ------------------------------------------------------------
# FIGURAS — Interarrival por MEDIDA (en unidades Tu)
# ------------------------------------------------------------

interarrival_measure_info = {}

for measure in df_measure_counts["measure"]:
    ev_ids = [
        ev_id for ev_id, name in inv_event.items()
        if extract_measure(name) == measure
    ]

    all_pos = []
    for ev_id in ev_ids:
        all_pos.extend(event_positions[ev_id].tolist())

    all_pos = np.array(sorted(all_pos), dtype=np.int64)

    if len(all_pos) < 3:
        interarrival_measure_info[measure] = {"has_fig": False, "reason": "<3 eventos"}
        continue

    epochs = df_events.iloc[all_pos]["segs"].values.astype(float)
    deltas = np.diff(epochs)
    deltas = deltas[deltas >= 0]

    if len(deltas) < 2:
        interarrival_measure_info[measure] = {"has_fig": False, "reason": "intervalos insuficientes"}
        continue

    deltas_TU = deltas / Tu
    clipped = np.clip(deltas_TU, 0, MAX_X_TU)

    fig_path = FIGURES_DIR / f"interarrival_measure_{measure}.png"

    def _plot():
        bins = min(MAX_BINS, len(np.unique(clipped)))
        plt.hist(clipped, bins=bins, edgecolor="black")
        plt.xlabel("Tiempo entre llegadas (Tu)")
        plt.ylabel("count")
        plt.title(f"Interarrival — medida {measure}")
        plt.xlim(0, MAX_X_TU)
        if np.any(deltas_TU > MAX_X_TU):
            plt.annotate("→", xy=(MAX_X_TU, plt.ylim()[1]*0.9),
                         color="red", fontsize=20, ha="center")

    save_figure(fig_path, plot_fn=_plot, figsize=(10, 4))

    interarrival_measure_info[measure] = {
        "has_fig": True,
        "file": fig_path.name,
        "num_events": int(len(all_pos)),
    }

print("[OK] Tablas y figuras de interarrival generadas")


[prepareeventsds] Generando tablas, estadísticas y figuras...


[OK] Tablas y figuras de interarrival generadas


In [12]:
# ============================================================
# 11.x — Informe HTML final (Fase 02)
# ============================================================

print("[prepareeventsds] Generando informe HTML final...")

REPORT_PATH = VARIANT_DIR / "02_prepareeventsds_report.html"

sections = []

# ------------------------------------------------------------
# Encabezado
# ------------------------------------------------------------

sections.append(f"<h1>PrepareEventsDS Report — Variante {ACTIVE_VARIANT}</h1>")
sections.append(f"<p><b>Fase padre:</b> {parent_phase}:{parent_variant}</p>")
sections.append(f"<p><b>Tu:</b> {Tu}</p>")
sections.append(f"<p><b>Band thresholds (%):</b> {band_thresholds_pct}</p>")
sections.append(f"<p><b>Event strategy:</b> {event_strategy}</p>")
sections.append(f"<p><b>NaN handling:</b> {nan_handling}</p>")
sections.append("<hr>")

# ------------------------------------------------------------
# Estadísticas globales
# ------------------------------------------------------------

total_events = len(flat_events)
epochs_with_events = (df_events["events"].apply(len) > 0).sum()

sections.append("<h2>Estadísticas globales</h2>")
sections.append(f"<p>Epochs totales: {len(df_events)}</p>")
sections.append(f"<p>Eventos totales: {total_events}</p>")
sections.append(f"<p>Tipos de evento distintos: {len(counts)}</p>")
sections.append(f"<p>Epochs con al menos un evento: {epochs_with_events}</p>")
sections.append(f"<p>Eventos medios por epoch: {total_events/len(df_events):.3f}</p>")

# ------------------------------------------------------------
# Tablas
# ------------------------------------------------------------

sections.append("<h2>Recuento por tipo de evento</h2>")
sections.append(df_event_counts.to_html(index=False))

sections.append("<h2>Recuento por medida</h2>")
sections.append(df_measure_counts.to_html(index=False))

sections.append("<h2>Estadísticas interarrival por evento</h2>")
sections.append(df_interarrival_stats.to_html(index=False))

# ------------------------------------------------------------
# Figuras interarrival por evento
# ------------------------------------------------------------

sections.append("<h2>Interarrival por evento (unidades Tu)</h2>")

for ev_id, info in interarrival_event_info.items():
    sections.append(f"<h3>Evento {ev_id}: {inv_event[ev_id]}</h3>")
    if not info["has_fig"]:
        sections.append(f"<p><i>No se genera figura: {info['reason']}</i></p>")
        continue
    sections.append(f"<p>Apariciones: {info['num_app']}</p>")
    sections.append(f"<img src='figures/{info['file']}' width='800'>")

# ------------------------------------------------------------
# Figuras interarrival por medida
# ------------------------------------------------------------

sections.append("<h2>Interarrival por medida (unidades Tu)</h2>")

for measure, info in interarrival_measure_info.items():
    sections.append(f"<h3>Medida {measure}</h3>")
    if not info["has_fig"]:
        sections.append(f"<p><i>No se genera figura: {info['reason']}</i></p>")
        continue
    sections.append(f"<p>Nº eventos asociados: {info['num_events']}</p>")
    sections.append(f"<img src='figures/{info['file']}' width='800'>")

# ------------------------------------------------------------
# Guardar HTML
# ------------------------------------------------------------

with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("<html><body>" + "\n".join(sections) + "</body></html>")

print(f"[OK] Informe HTML generado en {REPORT_PATH}")


[prepareeventsds] Generando informe HTML final...


[OK] Informe HTML generado en /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/02_prepareeventsds/v011/02_prepareeventsds_report.html
